In [85]:
import os, glob, io
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import numpy as np
from datetime import date
import unicodedata


# Ruta base (ajústala si cambia)
BASE_DIR = Path(r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G")

# Lista de 31 encabezados, en el orden en que los quieres (eNBId queda en W si mantienes este orden)
HEADERS = [
     "eNodeB Name","CellName","activePlmnList_mcc","additionalPlmnList_mcc",
    "administrativeState","cellBarred","cellId","cellSubscriptionCapacity",
    "channelSelectionSetSize","dlChannelBandwidth","earfcndl","earfcnul",
    "freqBand","noOfPucchCqiUsers","noOfPucchSrUsers","operationalState",
    "physicalLayerCellIdGroup","physicalLayerSubCellId","sectorCarrierRef",
    "tac","timeOfLastModification","ulChannelBandwidth",
    "eNBId","eNodeB Name Unique","LAT","LON","PCI","AT&T_Site_Name",
    "MOCN Activo por Celda","Al menos una celda de MOCN encendida","MME TEF"
]




In [86]:

def appendfiles(filenamepattern: str, save: bool = True) -> str | str:
    searchpattern = str(BASE_DIR / f"{filenamepattern}_*.txt")
    filestoread = glob.glob(searchpattern)
    print("Buscando:", searchpattern)
    print("Archivos:", filestoread)

    chunks = []
    for name in filestoread:
        with open(name, "r", encoding="utf-8") as f:
            chunks.append(f.read())
    integrated = "".join(chunks)

    if save:
        outputfile_name = f"Integrated_{filenamepattern}_files.txt"
        output_path = BASE_DIR / outputfile_name
        with open(output_path, "w", encoding="utf-8") as out:
            out.write(integrated)
        print("Integrado =>", outputfile_name)
        return outputfile_name
    else:
        print("Integrado en memoria (no se guardó archivo).")
        return integrated  # ← devuelve el TEXTO


def cleanfile(source: str, ignorelines=None, save: bool = True) -> str:
    """
    Si save=True, 'source' es NOMBRE de archivo. Si save=False, 'source' es TEXTO.
    """
    if ignorelines is None:
        ignorelines = ["SubNetwork,", "instance(s)", "NodeId"]

    if save:
        inputfile = BASE_DIR / source
        with open(inputfile, 'r', encoding="utf-8") as f_in:
            lines = f_in.readlines()
        kept = [ln for ln in lines if not any(p in ln for p in ignorelines)]
        cleanfile_name = f"Clean_{source}"
        with open(BASE_DIR / cleanfile_name, 'w', encoding="utf-8") as f_out:
            f_out.writelines(kept)
        print(f"Limpieza OK -> {cleanfile_name} ({len(kept)} líneas)")
        return cleanfile_name
    else:
        kept = []
        for line in source.splitlines(keepends=True):
            if any(p in line for p in ignorelines):
                continue
            kept.append(line)
        cleaned = "".join(kept)
        print(f"Limpieza en memoria OK -> líneas: {len(kept)}")
        return cleaned  # ← devuelve el TEXTO


def convert_to_excel(clean_source: str, save: bool = True) -> str | pd.DataFrame:
    """
    Si save=True, 'clean_source' es NOMBRE de archivo .txt; guarda un .xlsx.
    Si save=False, 'clean_source' es TEXTO; devuelve un DataFrame.
    """
    if save:
        cleanfile_path = BASE_DIR / clean_source
        out_xlsx = f"Converted_{clean_source}.xlsx"
        out_path = BASE_DIR / out_xlsx
        df = pd.read_csv(cleanfile_path, delimiter="\t", header=None)
        df.to_excel(out_path, index=False, header=None)
        print(f"Convertido a Excel -> {out_xlsx}  (shape={df.shape})")
        return out_xlsx
    else:
        df = pd.read_csv(io.StringIO(clean_source), delimiter="\t", header=None)
        print(f"Convertido a DataFrame en memoria (shape={df.shape})")
        return df  # ← devuelve el DF



In [87]:

# --- EUtranCellFDD en memoria ---
eu_txt   = appendfiles("EUtranCellFDD", save=False)   # texto crudo
eu_clean = cleanfile(eu_txt, save=False)              # texto limpio
df_eu    = convert_to_excel(eu_clean, save=False)     # DataFrame (no Excel)


# --- ENodeBFunction ---
nb_txt   = appendfiles('ENodeBFunction', save=False)     # string con contenido crudo
nb_clean = cleanfile(nb_txt, save=False)                # string limpio
df_nb    = convert_to_excel(nb_clean, save=False)       # DataFrame en memoria

# --- nodeid ---
nd_txt   = appendfiles('nodeid', save=False)
nd_clean = cleanfile(nd_txt, save=False)
df_nd    = convert_to_excel(nd_clean, save=False)

# --- MME ---
mme_txt   = appendfiles('MME', save=False)
mme_clean = cleanfile(mme_txt, save=False)
df_mme    = convert_to_excel(mme_clean, save=False)

# --- NbIotCell ---
nbiot_txt   = appendfiles('NbIotCell', save=False)
nbiot_clean = cleanfile(nbiot_txt, save=False)
df_nbiot    = convert_to_excel(nbiot_clean, save=False)




Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\EUtranCellFDD_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\EUtranCellFDD_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\EUtranCellFDD_9.txt']
Integrado en memoria (no se guardó archivo).
Limpieza en memoria OK -> líneas: 51830


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_3540\1924825416.py:67: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(clean_source), delimiter="\t", header=None)


Convertido a DataFrame en memoria (shape=(51814, 23))
Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\ENodeBFunction_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\ENodeBFunction_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\ENodeBFunction_9.txt']
Integrado en memoria (no se guardó archivo).
Limpieza en memoria OK -> líneas: 6953
Convertido a DataFrame en memoria (shape=(6937, 3))
Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\nodeid_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\nodeid_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\4G\\nodeid_9.txt']
Integrado en memoria (no se guardó archivo).
Limpieza en memoria OK -> líneas: 2658
Convertido a DataFrame en memoria (shape=(2648, 4))
Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\MME_*.txt
Archivos: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\

In [88]:

# %% Reacomodo de columnas en memoria
# df_eu es el DataFrame base que obtuviste de EUtranCellFDD (ya limpio)

cols = list(df_eu.columns)

# Guardamos nombre de la columna en posición B (índice 1)
col_B = cols[1]

# Resto de columnas a partir de la C
resto = cols[2:]

# Nuevo orden: primera columna (A), resto (C.), y al final la B
cols_reordered = [cols[0]] + resto + [col_B]

# Aplicar el reordenamiento
df_base = df_eu[cols_reordered]

print("Reacomodo OK -> DataFrame en memoria con columnas reordenadas:", df_base.shape)




Reacomodo OK -> DataFrame en memoria con columnas reordenadas: (51814, 23)


In [90]:


# Ya tienes df_base (reacomodado y sin headers reales todavía)

# Verificación de columnas
n_cols = df_base.shape[1]
print("Columnas detectadas en df_base (reacomodado):", n_cols)

# %% Ajuste de headers en memoria (Opción B - reindex)

# Aseguramos que df_base sea copia real (evita warnings de "view")
df_base = df_base.copy()

# Forzar que tenga exactamente las columnas de HEADERS:
# - Si falta alguna, la agrega rellena con NA.
# - Si sobra alguna, la elimina.
# - Ordena exactamente como HEADERS.
df_base = df_base.reindex(columns=HEADERS, fill_value=pd.NA)

print("Headers asignados y ordenados con reindex:", df_base.shape)








Columnas detectadas en df_base (reacomodado): 31
Headers asignados y ordenados con reindex: (51814, 31)


In [91]:
# %% [A] Alinear df_base exactamente a HEADERS (sin Excel)
df_base = df_base.copy()
df_base = df_base.reindex(columns=HEADERS, fill_value=pd.NA)
print("df_base listo:", df_base.shape)

# %% [B] NbIotCell en memoria (df_nbiot ya creado previamente)
# Validación de columnas mínimas (A..N => 14)
if df_nbiot.shape[1] < 14:
    raise ValueError("NbIotCell debe tener al menos 14 columnas (A..N).")

# Tomar solo A.N
df_nbiot14 = df_nbiot.iloc[:, :14].copy()

# %% [C] Construir bloque NbIot alineado por POSICIÓN (mapeo requerido)
# Mapeo (según tu comentario original):
# A->A, C->B, D->C, E->E, F->F, G->G, H->K, I->L, J->P, K->R, L->D, M->S, N->T
df_nb = pd.DataFrame(pd.NA, index=df_nbiot14.index, columns=HEADERS)

df_nb["eNodeB Name"]                 = df_nbiot14.iloc[:, 0]    # A -> A
# (OJO: ya NO mapeamos ENodeBFunctionId para que no aparezca en el header)
df_nb["CellName"]                    = df_nbiot14.iloc[:, 2]    # C -> B
df_nb["activePlmnList_mcc"]          = df_nbiot14.iloc[:, 3]    # D -> C
df_nb["administrativeState"]         = df_nbiot14.iloc[:, 4]    # E -> E
df_nb["cellBarred"]                  = df_nbiot14.iloc[:, 5]    # F -> F
df_nb["cellId"]                      = df_nbiot14.iloc[:, 6]    # G -> G
df_nb["earfcndl"]                    = df_nbiot14.iloc[:, 7]    # H -> K
df_nb["earfcnul"]                    = df_nbiot14.iloc[:, 8]    # I -> L
df_nb["operationalState"]            = df_nbiot14.iloc[:, 9]    # J -> P
df_nb["physicalLayerCellIdGroup"]    = df_nbiot14.iloc[:,10]    # K -> R
df_nb["additionalPlmnList_mcc"]      = df_nbiot14.iloc[:,11]    # L -> D
df_nb["sectorCarrierRef"]            = df_nbiot14.iloc[:,12]    # M -> S
df_nb["tac"]                         = df_nbiot14.iloc[:,13]    # N -> T

# %% [D] Unir debajo y listo (sin guardar a disco)
df_out = pd.concat([df_base, df_nb], ignore_index=True)

print("OK: NbIotCell agregado debajo (en memoria). Shape final:", df_out.shape)


df_base listo: (51814, 31)
OK: NbIotCell agregado debajo (en memoria). Shape final: (59331, 31)


In [93]:



# ============ eNBId desde ENodeBFunction (DF en memoria) ============
# df_enbfun: sin headers (como viene del TXT). Tomamos primeras 3 columnas.
df_nodeb = df_nb.iloc[:, :3].copy()
df_nodeb.columns = ["NodeId", "ENodeBFunctionId", "eNBIdnew"]

# normaliza tipos/espacios
df_out["eNodeB Name"] = df_out["eNodeB Name"].astype(str).str.strip()
df_nodeb["NodeId"] = df_nodeb["NodeId"].astype(str).str.strip()

# dedupe por NodeId
df_nodeb = df_nodeb.drop_duplicates(subset=["NodeId"], keep="first")

# LEFT JOIN por eNodeB Name (df_out) = NodeId (df_nodeb)
df_tmp = df_out.merge(
    df_nodeb[["NodeId", "eNBIdnew"]],
    left_on="eNodeB Name",
    right_on="NodeId",
    how="left"
)
df_out["eNBId"] = df_tmp["eNBIdnew"]

# ============ eNodeB Name Unique (solo cuando cambia) ============
_name = df_out["eNodeB Name"].astype(str).fillna("").str.strip()
is_new = _name.ne(_name.shift())
df_out["eNodeB Name Unique"] = np.where(is_new & _name.ne(""), df_out["eNodeB Name"], "")

# ============ LAT/LON/AT&T_Site_Name desde All_Ericsson_4G_{YYYYMM-1} ============
today = date.today()
prev_year  = today.year if today.month > 1 else today.year - 1
prev_month = today.month - 1 or 12
yyyymm_prev = f"{prev_year}{prev_month:02d}"

ae_path = BASE_DIR / f"All_Ericsson_4G_{yyyymm_prev}.xlsx"
ae_df = pd.read_excel(ae_path, usecols=["eNodeB Name", "LAT", "LON", "AT&T_Site_Name"])
ae_df["eNodeB Name"] = ae_df["eNodeB Name"].astype(str).str.strip()
ae_df = ae_df.drop_duplicates(subset=["eNodeB Name"], keep="first")

merged = df_out.merge(ae_df, on="eNodeB Name", how="left", suffixes=("", "_ae"))

def _is_blank(s: pd.Series) -> pd.Series:
    return s.isna() | s.astype(str).str.strip().eq("")

for col in ["LAT", "LON", "AT&T_Site_Name"]:
    m = _is_blank(merged[col])
    if f"{col}_ae" in merged:
        merged.loc[m, col] = merged.loc[m, f"{col}_ae"]
        merged.drop(columns=[f"{col}_ae"], inplace=True, errors="ignore")

merged = merged.infer_objects(copy=False)

# ======= Fallback EPT por "Cell Name" (solo donde aún falte) =======
faltan = _is_blank(merged["LAT"]) | _is_blank(merged["LON"]) | _is_blank(merged["AT&T_Site_Name"])
if not faltan.any():
    df_out = merged
    print("All_Ericsson cubrió 100% (LAT/LON/AT&T_Site_Name). Se omite fallback EPT.")
else:
    print(f"Quedan {int(faltan.sum())} filas con faltantes. Se aplica fallback EPT…")
    ept_matches = glob.glob(str(BASE_DIR / "EPT_ATT_UMTS_LTE_*.xlsx"))
    if ept_matches:
        ept_file = ept_matches[0]
        ept_sheets = ["EPT_3G_LTE_OUTDOOR", "PLAN_OUTDOOR", "EPT_3G_LTE_INDOOR", "PLAN_INDOOR", "Eventos_Especiales"]

        frames = []
        for sh in ept_sheets:
            try:
                tmp = pd.read_excel(ept_file, sheet_name=sh, engine="openpyxl")
                frames.append(tmp)
            except Exception:
                pass

        def coalesce_to_single_column(df, variants, target):
            present = [c for c in variants if c in df.columns]
            if not present:
                return df
            cols = [target] + [c for c in present if c != target] if target in present else present
            merged_series = df[cols].bfill(axis=1).iloc[:, 0]
            df[target] = merged_series
            df.drop(columns=[c for c in present if c != target], inplace=True, errors="ignore")
            return df

        if frames:
            ept_df = pd.concat(frames, ignore_index=True)

            # Unificar nombres de columnas
            ept_df = coalesce_to_single_column(ept_df, ["Cell Name", "CellName", "ATT_CELL_ID_Name"], "Cell Name")
            ept_df = coalesce_to_single_column(ept_df, ["Latitud"], "LAT")
            ept_df = coalesce_to_single_column(ept_df, ["Longitud"], "LON")
            ept_df = coalesce_to_single_column(ept_df, ["AT&T_Site_Name"], "AT&T_Site_Name")

            # LAT/LON a float (evita FutureWarning)
            for c in ["LAT", "LON"]:
                if c in ept_df.columns:
                    ept_df[c] = (
                        ept_df[c].astype(str).str.strip("[]").str.replace(",", "", regex=False)
                    )
                    ept_df[c] = pd.to_numeric(ept_df[c], errors="coerce")

            # Clean strings
            for c in ["Cell Name", "AT&T_Site_Name"]:
                if c in ept_df.columns:
                    ept_df[c] = ept_df[c].astype(str).str.strip()

            ept_df = ept_df.drop_duplicates(subset=["Cell Name"], keep="first")

            # Asegurar llave en base
            created_temp_key = False
            if "Cell Name" not in merged.columns:
                if "CellName" in merged.columns:
                    merged["Cell Name"] = merged["CellName"]
                    created_temp_key = True
                else:
                    raise KeyError("No se encontró 'Cell Name' ni 'CellName' en la base para hacer join con EPT.")

            merged["Cell Name"] = merged["Cell Name"].astype(str).str.strip()

            # Merge y completar SOLO faltantes
            ept_lookup = ept_df[["Cell Name", "LAT", "LON", "AT&T_Site_Name"]].rename(
                columns={"LAT": "LAT_ept", "LON": "LON_ept", "AT&T_Site_Name": "AT&T_Site_Name_ept"}
            )
            merged = merged.merge(ept_lookup, on="Cell Name", how="left")

            need_lat  = _is_blank(merged["LAT"])
            need_lon  = _is_blank(merged["LON"])
            need_site = _is_blank(merged["AT&T_Site_Name"])

            if "LAT_ept" in merged: merged.loc[need_lat,  "LAT"] = merged.loc[need_lat,  "LAT_ept"]
            if "LON_ept" in merged: merged.loc[need_lon,  "LON"] = merged.loc[need_lon,  "LON_ept"]
            if "AT&T_Site_Name_ept" in merged: merged.loc[need_site, "AT&T_Site_Name"] = merged.loc[need_site, "AT&T_Site_Name_ept"]

            merged.drop(columns=[c for c in ["LAT_ept","LON_ept","AT&T_Site_Name_ept"] if c in merged.columns], inplace=True)
            if created_temp_key:
                merged.drop(columns=["Cell Name"], inplace=True)

            print("Fallback EPT aplicado por 'Cell Name':", ept_file)
        else:
            print("No se encontraron archivos EPT_ATT_UMTS_LTE_*.xlsx; se omite fallback.")

    df_out = merged  # resultado tras fallback (o no)

# ============ Calcular PCI = IF(R blank, Q, Q*3 + R) ============
q_col = "physicalLayerCellIdGroup"
r_col = "physicalLayerSubCellId"

df_out[q_col] = pd.to_numeric(df_out[q_col], errors="coerce")
df_out[r_col] = pd.to_numeric(df_out[r_col].astype(str).str.strip().replace({"": None}), errors="coerce")

df_out["PCI"] = pd.Series(
    np.where(df_out[r_col].isna(), df_out[q_col], df_out[q_col]*3 + df_out[r_col]),
    index=df_out.index,
    dtype="Int64"
)

# ============ MOCN ============
pattern = "[{mncLength=3, mcc=334, mnc=90}, {mncLength=2, mcc=334, mnc=3}, {mncLength=2, mcc=1, mnc=1}, {mncLength=2, mcc=1, mnc=1}, {mncLength=2, mcc=1, mnc=1}]"
df_out["MOCN Activo por Celda"] = np.where(
    df_out["additionalPlmnList_mcc"].astype(str).str.strip() == pattern, "Si", "No"
)

truthy = {"si", "sí", "yes", "true", "1"}
enb = df_out["eNodeB Name"].astype(str).str.strip()
mocn = df_out["MOCN Activo por Celda"].astype(str).str.strip().str.lower()
enbs_con_mocn = set(enb[mocn.isin(truthy)])
df_out["Al menos una celda de MOCN encendida"] = np.where(enb.isin(enbs_con_mocn), "Si", "No")

# ============ MME TEF (DF en memoria) ============
# df_mme: sin headers. Tomamos 3 primeras columnas y nombramos.
df_MME = df_mme.iloc[:, :3].copy()
df_MME.columns = ["NodeId", "eNodeBFunction", "TermPointToMmeId"]
df_MME["NodeId"] = df_MME["NodeId"].astype(str).str.strip()

# Filtrar NodeId de longitud 7 (según tu requerimiento)
df_MME_7 = df_MME[df_MME["NodeId"].str.len() == 7].copy()

# Conteo por NodeId
mme_counts = df_MME_7["NodeId"].value_counts()

# Mapear conteo a tu base por eNodeB Name
df_out["eNodeB Name"] = df_out["eNodeB Name"].astype(str).str.strip()
df_out["MME TEF"] = df_out["eNodeB Name"].map(mme_counts).fillna(0).astype("Int64")

# ============ Guardado final con nombre dinámico (si quieres escribir) ============
today = date.today()
yyyymm = f"{today.year}{today.month:02d}"   # p.ej. 202509
final_path = BASE_DIR / f"All_Ericsson_4G_{yyyymm}.xlsx"

# Si quieres guardar aquí:
# df_out.to_excel(final_path, index=False)
print("Reporte final listo →", final_path, "shape=", df_out.shape)






Quedan 1 filas con faltantes. Se aplica fallback EPT…
Fallback EPT aplicado por 'Cell Name': C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\EPT_ATT_UMTS_LTE_2025-08-14.xlsx
Reporte final listo → C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\4G\All_Ericsson_4G_202509.xlsx shape= (59331, 31)


In [94]:


today = date.today()
yyyymm = f"{today.year}{today.month:02d}"   # ej. 202509

final_excel = BASE_DIR / f"All_Ericsson_4G_{yyyymm}.xlsx"
tmp_excel   = BASE_DIR / f"~tmp_All_Ericsson_4G_{yyyymm}.xlsx"

# Releer, forzar columnas y orden
df_out = pd.read_excel(final_excel)

# Garantiza que TODAS las columnas existan
for col in HEADERS:
    if col not in df_out.columns:
        df_out[col] = pd.NA

# Reordena exactamente como HEADERS
df_out = df_out[HEADERS]

# Escribe temporal
df_out.to_excel(tmp_excel, index=False)

# Reaplicar estilo vertical de headers
wb = load_workbook(tmp_excel)
ws = wb.active

# Congelar encabezado
ws.freeze_panes = "A2"

# Aplicar estilo a fila 1
for col_idx, header in enumerate(HEADERS, start=1):
    cell = ws.cell(row=1, column=col_idx)
    cell.value = header
    cell.font = Font(name="Aptos Narrow", size=11)  # bold=True si quieres negrita
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="bottom", wrap_text=True)

wb.save(final_excel)

# Limpia temporal
try:
    tmp_excel.unlink()
except Exception as e:
    print("No se pudo borrar temporal:", e)

print("Ajuste final OK -> Headers verticales y columnas forzadas/ordenadas.")



Ajuste final OK -> Headers verticales y columnas forzadas/ordenadas.
